## 嵌入

在我們之前的例子中，我們操作的是長度為 `vocab_size` 的高維度詞袋向量，並且我們將低維度的位置表示向量顯式地轉換為稀疏的獨熱表示（one-hot representation）。這種獨熱表示並不具備記憶效率。此外，每個詞彙都被獨立處理，因此獨熱編碼向量無法表達詞彙之間的語義相似性。

在本單元中，我們將繼續探索 **News AG** 數據集。首先，讓我們載入數據並從上一單元中獲取一些定義。


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### 什麼是嵌入？

**嵌入**的概念是用低維度的密集向量來表示詞語，這些向量反映了詞語的語義。稍後我們會討論如何構建有意義的詞嵌入，但現在可以簡單地將嵌入理解為一種降低詞向量維度的方法。

嵌入層以詞語作為輸入，並生成指定的 `embedding_size` 的輸出向量。某種程度上，它與 `Dense` 層非常相似，但嵌入層不需要以 one-hot 編碼向量作為輸入，而是可以直接接受詞語編號。

通過在網絡中使用嵌入層作為第一層，我們可以從詞袋模型切換到 **嵌入袋** 模型。在嵌入袋模型中，我們首先將文本中的每個詞轉換為相應的嵌入，然後對所有嵌入進行某種聚合函數計算，例如 `sum`、`average` 或 `max`。

![展示五個序列詞嵌入分類器的圖片。](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

我們的分類器神經網絡包含以下幾層：

* `TextVectorization` 層：該層以字符串作為輸入，並生成包含詞語編號的張量。我們會指定一個合理的詞彙表大小 `vocab_size`，並忽略使用頻率較低的詞語。輸入形狀為 1，輸出形狀為 $n$，因為結果中會有 $n$ 個標記，每個標記包含從 0 到 `vocab_size` 的數字。
* `Embedding` 層：該層接收 $n$ 個數字，並將每個數字縮減為指定長度的密集向量（在我們的例子中是 100）。因此，形狀為 $n$ 的輸入張量將被轉換為 $n\times 100$ 的張量。
* 聚合層：該層沿第一軸計算該張量的平均值，即計算所有 $n$ 個輸入張量（對應不同詞語）的平均值。為了實現該層，我們會使用 `Lambda` 層，並傳入計算平均值的函數。輸出形狀為 100，這將是整個輸入序列的數字表示。
* 最後的 `Dense` 線性分類器。


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


在 `summary` 的輸出中，**output shape** 欄位中的第一個張量維度 `None` 代表小批量的大小，第二個維度則代表標記序列的長度。小批量中的所有標記序列長度都不相同。我們會在下一節討論如何處理這個問題。

現在讓我們開始訓練網絡：


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **注意** 我們正在基於數據的一個子集構建向量化器。這樣做是為了加快過程，但可能會導致我們文本中的某些詞彙未出現在詞彙表中。在這種情況下，這些詞彙將被忽略，這可能會導致準確性略有下降。然而，在現實生活中，文本的子集通常能提供良好的詞彙估計。


### 處理變量序列長度

讓我們了解小批量訓練的過程。在上述例子中，輸入張量的維度是1，我們使用長度為128的小批量，因此張量的實際大小是 $128 \times 1$。然而，每個句子的詞元數量是不同的。如果我們將 `TextVectorization` 層應用到單一輸入，返回的詞元數量會根據文本如何被分詞而有所不同：


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


然而，當我們將向量化器應用於多個序列時，它必須生成一個矩形形狀的張量，因此會用 PAD 標記（在我們的情況下是零）填充未使用的元素：


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

在這裡我們可以看到嵌入：


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **注意**：為了減少填充的數量，在某些情況下，將數據集中所有序列按長度（更準確地說，是按標記數量）遞增的順序排序是有意義的。這將確保每個小批次包含長度相似的序列。


## 語義嵌入：Word2Vec

在之前的例子中，嵌入層學習如何將文字映射到向量表示，但這些表示並不具有語義上的意義。如果能學習一種向量表示，使得相似的詞或同義詞在某種向量距離（例如歐幾里得距離）上彼此接近，那就更好了。

為了達到這個目的，我們需要使用像 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) 這樣的技術，在大量文本集合上預訓練嵌入模型。Word2Vec 基於兩種主要架構，用於生成詞的分佈式表示：

 - **連續詞袋模型** (CBoW)，我們訓練模型根據周圍的上下文來預測一個詞。給定 ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$，模型的目標是根據 $(W_{-2},W_{-1},W_1,W_2)$ 預測 $W_0$。
 - **連續跳字模型** (Skip-Gram) 與 CBoW 相反。模型使用周圍的上下文詞窗口來預測當前的詞。

CBoW 的速度較快，而 Skip-Gram 雖然較慢，但在表示不常見的詞方面效果更好。

![展示 CBoW 和 Skip-Gram 算法如何將詞轉換為向量的圖片。](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

為了試驗基於 Google News 數據集預訓練的 Word2Vec 嵌入，我們可以使用 **gensim** 庫。以下是找到與「neural」最相似的詞的示例。

> **注意：** 當你第一次創建詞向量時，下載它們可能需要一些時間！


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


我們還可以從該詞中提取向量嵌入，用於訓練分類模型。該嵌入有300個組件，但這裡為了清晰起見，我們僅顯示向量的前20個組件：


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

語義嵌入的偉大之處在於你可以根據語義操控向量編碼。例如，我們可以要求找到一個詞，其向量表示盡可能接近詞語*國王*和*女人*，並且盡可能遠離詞語*男人*：


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

上面的例子使用了一些內部的 GenSym 魔法，但其基本邏輯其實相當簡單。關於嵌入的一個有趣之處是，你可以對嵌入向量執行正常的向量運算，而這將反映在詞語**意義**上的操作。上面的例子可以用向量運算來表達：我們計算出對應於 **KING-MAN+WOMAN** 的向量（對應詞語的向量表示執行 `+` 和 `-` 操作），然後在字典中找到最接近該向量的詞語：


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **注意**：我們需要在 *man* 和 *woman* 向量中加入一個小係數——試試移除它們，看看會發生什麼。

為了找到最接近的向量，我們使用 TensorFlow 的工具來計算我們的向量與詞彙表中所有向量之間的距離向量，然後使用 `argmin` 找出最小距離的詞的索引。


雖然 Word2Vec 是一種表達詞語語義的好方法，但它有許多缺點，包括以下幾點：

* CBoW 和 skip-gram 模型都是**預測型嵌入**，它們只考慮局部上下文。Word2Vec 無法利用全局上下文。
* Word2Vec 沒有考慮到詞語的**形態學**，即詞語的意義可能取決於詞的不同部分，例如詞根。

**FastText** 嘗試克服第二個限制，並在 Word2Vec 的基礎上進一步改進，通過學習每個詞的向量表示以及詞內的字符 n-gram。這些表示的值在每次訓練步驟中會被平均成一個向量。雖然這為預訓練增加了大量額外的計算，但它使詞嵌入能夠編碼子詞資訊。

另一種方法，**GloVe**，採用了與 Word2Vec 不同的詞嵌入方法，基於詞-上下文矩陣的分解。首先，它構建了一個大型矩陣，記錄詞語在不同上下文中的出現次數，然後嘗試以降低維度的方式表示該矩陣，同時最小化重建損失。

gensim 庫支持這些詞嵌入方法，您可以通過修改上方的模型加載代碼來嘗試使用它們。


## 在 Keras 中使用預訓練的嵌入

我們可以修改上述範例，在嵌入層的矩陣中預先填入語義嵌入，例如 Word2Vec。預訓練嵌入的詞彙表和文本語料庫的詞彙表可能不一致，因此我們需要選擇其中一個。在這裡，我們探討兩種可能的選擇：使用分詞器的詞彙表，或者使用 Word2Vec 嵌入的詞彙表。

### 使用分詞器的詞彙表

當使用分詞器的詞彙表時，詞彙表中的某些詞會有對應的 Word2Vec 嵌入，而某些則會缺失。假設我們的詞彙表大小為 `vocab_size`，而 Word2Vec 嵌入向量的長度為 `embed_size`，那麼嵌入層將由一個形狀為 `vocab_size`$\times$`embed_size` 的權重矩陣表示。我們將通過遍歷詞彙表來填充這個矩陣：


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


對於不在 Word2Vec 詞彙表中的詞語，我們可以選擇將它們設為零向量，或者生成一個隨機向量。

現在我們可以定義一個帶有預訓練權重的嵌入層：


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **注意**：請注意，我們在建立 `Embedding` 時設置了 `trainable=False`，這表示我們不會重新訓練 Embedding 層。這可能會導致準確度稍微降低，但能加快訓練速度。

### 使用嵌入詞彙表

之前方法的一個問題是，TextVectorization 和 Embedding 使用的詞彙表不同。為了解決這個問題，我們可以採用以下其中一種解決方案：
* 重新訓練 Word2Vec 模型以適配我們的詞彙表。
* 使用預訓練 Word2Vec 模型的詞彙表來載入我們的數據集。在載入數據集時，可以指定使用的詞彙表。

第二種方法看起來更簡單，所以我們來實現它。首先，我們將使用 Word2Vec 嵌入中指定的詞彙表來建立一個 `TextVectorization` 層：


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

gensim 的詞嵌入庫包含一個方便的函數 `get_keras_embeddings`，它會自動為你創建相應的 Keras 嵌入層。


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


我們未能看到更高準確度的其中一個原因是因為我們的數據集中的某些詞語在預訓練的GloVe詞彙表中缺失，因此它們基本上被忽略了。為了解決這個問題，我們可以基於我們的數據集訓練自己的嵌入。


## 語境嵌入

傳統預訓練嵌入表示（例如 Word2Vec）的一個主要限制是，雖然它們可以捕捉到某些詞語的含義，但無法區分不同的意思。這可能會在下游模型中引起問題。

例如，單詞「play」在以下兩個句子中有不同的意思：
- 我去劇院看了一場**戲劇**。
- John 想和他的朋友一起**玩**。

我們提到的預訓練嵌入會將「play」這個詞的兩種意思表示為相同的嵌入。為了解決這個限制，我們需要基於**語言模型**來構建嵌入，語言模型是在大量文本語料上訓練的，並且*知道*單詞如何在不同語境中組合使用。討論語境嵌入超出了本教程的範圍，但我們會在下一單元討論語言模型時回到這個主題。



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原文文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
